# Figure S1: Glacier projections by hydrological zones

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl     = px.colors.qualitative.D3
os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

# Ids for each glacier

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 1][["RGIId", "Zone"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 1]

# RGI6 doesnt have IDs yet 
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone'})

RGI7_ids = RGI7_ids[["RGIId", "Zone"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

In [ ]:
def preprocess(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])[variables]

def postprocessing(ds, scenario): # clean dataframe
    ds = ds.to_dataframe()
    ds["scenario"] = scenario
    ds = ds.set_index("scenario", append=True)
    ds = ds.reorder_levels(['scenario', 'rgi_id', 'time'])
    return ds

In [ ]:
# variables to analize
variables        = ['volume', 'area']
scenarios        = ["ct_random","ssp126","ssp245","ssp370","ssp585"]

In [ ]:
# historical period
all_combs = glob("/home/rooda/OGGM_results/new/*/run_outputs_*.nc", recursive = True)
all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

# assing zone to each glacier and aggregate the result 
ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
all_opts   = all_opts.groupby('rgi_id').sum()
all_opts   = all_opts.chunk("auto")
all_opts["smb"] = (all_opts.volume.diff(dim = "time") * 900 / all_opts.area) 
all_opts   = all_opts.isel(time = slice(0, -1))

# standard desviation
dataset_var_hist = all_opts.std(dim="options")
dataset_var_hist = postprocessing(dataset_var_hist, "historical")

# mean
dataset_mean_hist = all_opts.mean(dim="options")
dataset_mean_hist = postprocessing(dataset_mean_hist, "historical")

In [ ]:
dataset_mean = []
dataset_var  = []

for scenario in tqdm(scenarios): 
    all_combs = glob("/home/rooda/OGGM_results/new/*/run_output_*"+ scenario +"*.nc", recursive = True)
    all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

    if scenario == "ct_random":
        all_opts["time"] = all_opts["time"] + 2020
    
    # assing zone to each glacier and aggregate the result 
    ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
    all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
    all_opts   = all_opts.groupby('rgi_id').sum()
    all_opts   = all_opts.chunk("auto")
    all_opts["smb"] = (all_opts.volume.diff(dim = "time") * 900 / all_opts.area) 
    all_opts   = all_opts.isel(time = slice(0, -1))
    
    # standard desviation
    all_opts_var = all_opts.std(dim="options")
    all_opts_var = postprocessing(all_opts_var, scenario)
    dataset_var.append(all_opts_var)
    
    # mean
    all_opts_mean = all_opts.mean(dim="options")
    all_opts_mean = postprocessing(all_opts_mean, scenario)
    dataset_mean.append(all_opts_mean)
    
dataset_mean = pd.concat(dataset_mean)
dataset_mean = pd.concat([dataset_mean_hist, dataset_mean]).reset_index()

dataset_var  = pd.concat(dataset_var)
dataset_var  = pd.concat([dataset_var_hist, dataset_var]).reset_index()

In [ ]:
# normalize volume and area 
dataset_mean_ref = dataset_mean[dataset_mean.time == 2015]

In [ ]:
dict_replace = {"scenario": {"historical":'Historical', "ct_random":'Commitment run', "ssp126":'SSP 126', "ssp245":'SSP 245', "ssp370":'SSP 370', "ssp585":'SSP 585'}}

dataset_mean = dataset_mean.replace(dict_replace)
dataset_var  = dataset_var.replace(dict_replace)
dataset_mean_ref = dataset_mean_ref.replace(dict_replace)

In [ ]:
scenarios   = ["Historical", "Commitment run", "SSP 126", "SSP 245", "SSP 370", "SSP 585"]
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", "Commitment run":"rgba(0, 0, 0, 0.5)", "SSP 126":cl[0], "SSP 245":cl[2], "SSP 370":cl[1], "SSP 585":cl[3]}

basins_id  = ['PPY', 'PCA','NPI-E','NPI-W','SPI-N', 'SPI-C', 'SPI-S', 'GCN', 'CDI']


fig    = make_subplots(rows=9, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.01, 
                       shared_xaxes= True, shared_yaxes= False, row_titles = basins_id,
                       subplot_titles= ["Glacier volume (%)", "Glacier area (%)", "Specific mass balance (kg m<sup>-2</sup>)"])

for x in range(0,9):
    for y in range(0,3):
        for t in range(0,6):
             
            # time series for each subplot and scenario
            time_series_id    = dataset_mean[dataset_mean.rgi_id == basins_id[x]][dataset_mean.scenario == scenarios[t]]
            time_series_sd_id = dataset_var[dataset_var.rgi_id == basins_id[x]][dataset_var.scenario == scenarios[t]]    
            ts_ref_id         = dataset_mean_ref[dataset_mean_ref.rgi_id == basins_id[x]]
            
            if x==0 and y==0: # legend only for first plot
        
                fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3]/ts_ref_id.iloc[0,y+3], 
                                         mode='lines', name= scenarios[t], 
                                         marker=dict(color=scen_colors[scenarios[t]]), showlegend=True, legendgroup=t), row=x+1, col=y+1)

                fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                         line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=x+1, col=y+1)
                fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                         line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)
                
            else:
                if y==2: # dont normalize specific mass balance
            
                    fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3], mode='lines', name= scenarios[t], 
                                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=x+1, col=y+1)

                    # uncertainty: +-1 sd
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup=t), row=x+1, col=y+1)
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)
       
                else:
                    # mean value
                    fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3]/ts_ref_id.iloc[0,y+3], mode='lines', name= scenarios[t], 
                                             marker=dict(color=scen_colors[scenarios[t]]), showlegend=False, legendgroup=t), row=x+1, col=y+1)

                    # uncertainty: +-1 sd
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup=t), row=x+1, col=y+1)
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3])/ts_ref_id.iloc[0,y+3], 
                                             line=dict(width=0), fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

#  some tweaks
for x in range(0,9):
    for y in range(0,3):                
        if y==2: fig.update_yaxes(dtick = 2000,  row=x+1, col=y+1)
        else: fig.update_yaxes(range = [0, 1.2], dtick = 0.5, tickformat=".0%", row=x+1, col=y+1)
        
fig.update_yaxes(range = [-5500, 500], dtick = 3000, row=5, col=3)

fig.update_yaxes(ticks="outside", griddash = "dot", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, tickangle = -90)
fig.update_xaxes(ticks="outside", griddash = "dot", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=-0.02, xanchor="left", x=0.15, orientation="h", bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=1200, width=1000, template = "seaborn", margin = dict(l=20, r=20, b=30, t=30), hovermode = False)
fig.update_layout(plot_bgcolor = "rgba(213,213,213,0.6)")

# save figure 
#fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_S1_Glacier_projections.png", scale=4)
fig.show()

In [ ]:
# regional loss
dataset_mean[dataset_mean.rgi_id == "NPI-E"][dataset_mean.scenario == scenarios[2]].groupby("time").sum().smb.iloc[50:].mean()
dataset_mean[dataset_mean.rgi_id == "NPI-E"][dataset_mean.scenario == scenarios[2]].groupby("time").sum().smb.iloc[50:].std()

dataset_mean[dataset_mean.rgi_id == "NPI-E"][dataset_mean.scenario == scenarios[5]].groupby("time").sum().smb.iloc[50:].mean()
dataset_mean[dataset_mean.rgi_id == "NPI-E"][dataset_mean.scenario == scenarios[5]].groupby("time").sum().smb.iloc[50:].std()